In [9]:
%pip install --upgrade uszipcode

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy as gp
import uszipcode as zp

/Users/sergioferreira/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [30]:
#from uszipcode import Zipcode
# Search zipcode within 30 miles, ordered from closest to farthest
search = zp.SearchEngine()
result = search.by_coordinates(39.122229, -77.133578, radius=10, returns=1)
test = result[0].zipcode
type(test)

str

In [2]:
#df = pd.read_csv('inside_airbnb_zip.csv.gz', index_col = False, compression='gzip', low_memory = False)
copy_cols1 = ['id',
#            'scrape_id',
            'last_scraped',
            'host_id',
            'host_name',
#            'host_since',
#            'host_location',
            'calculated_host_listings_count',
#            'street',
            'neighbourhood_cleansed',
            'neighbourhood_group_cleansed',
#            'state',
#            'zipcode',
#            'smart_location',
#            'country',
            'latitude',
            'longitude',
#            'property_type',
            'room_type',
#            'accommodates',
#            'bathrooms',
            'bedrooms',
            'availability_365']

copy_cols2 = ['id',
            'host_id',
            'host_name',
            'calculated_host_listings_count',
            'neighbourhood',
            'neighbourhood_group',
            'latitude',
            'longitude',
            'room_type',
            'availability_365']

rename_cols2 = ['id',
            'host_id',
            'host_name',
            'calculated_host_listings_count',
            'neighbourhood_cleansed',
            'neighbourhood_group_cleansed',
            'latitude',
            'longitude',
            'room_type',
            'availability_365']

read1 = ['data/abnb_9-2022.gz','data/abnb_6-2022.gz',
        'data/abnb_3-2022.gz','data/abnb_11-2021.csv',
         'data/abnb_2015.csv','data/abnb_2016.csv',
         'data/abnb_2017.csv','data/abnb_2018.csv',
         'data/abnb_2019.csv','data/abnb_2020.csv',]

#df = pd.read_csv(, skipinitialspace=True, usecols=copy_cols, low_memory = False)

counter = 0
for item in read1:
    counter += 1
    x=0
    x2=0
    y=0
    y2=0
    if item == read1[0]:
        df1 = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        df1=df1.drop_duplicates(subset='id')
#        df = df.rename(columns = rename_cols)
    elif item in read1[1:4]: #change to 4 for 2021 csv
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
#        df2=df2.rename(columns = rename_cols2)
        df1 = pd.concat([df1, temp] , sort=False)
#        df1=df1.drop_duplicates(subset='id')
    elif item == read1[4]:
        df2 = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols2, sep=',', low_memory = False)
        df2=df2.drop_duplicates(subset='id')
        y_ins = item[10:14]
        df2['last_scraped'] = y_ins
    elif item in read1[5:10]:
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols2, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
        y_ins = item[10:14]
        temp['last_scraped'] = y_ins
        df2 = pd.concat([df2, temp], sort=False)
    
#df2['zipcode'] = 'N/A'
df2['bedrooms'] = 'N/A'
df2 = df2.rename(columns={'neighbourhood_group':'neighbourhood_group_cleansed', 'neighbourhood': 'neighbourhood_cleansed'})
df2 = df2[['id', 'last_scraped', 'host_id', 'host_name', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude', 'room_type',
       'bedrooms', 'availability_365', 'calculated_host_listings_count']]

In [3]:
df1 = pd.concat([df1,df2], sort=False)

In [4]:
df1 = df1.rename(columns={'neighbourhood_group_cleansed': 'borough','neighbourhood_cleansed':'neighbourhood'})

In [5]:
df1['zipcode'] = 'N/A'

In [9]:
df1.keys()

Index(['id', 'last_scraped', 'host_id', 'host_name', 'neighbourhood',
       'borough', 'latitude', 'longitude', 'room_type', 'bedrooms',
       'availability_365', 'calculated_host_listings_count', 'zipcode'],
      dtype='object')

In [14]:
lat = list(df1['latitude'])
lng = list(df1['longitude'])

In [41]:
lat_df = pd.DataFrame(lat, columns=["latitude"])
lng_df = pd.DataFrame(lng, columns=["longitude"])

In [44]:
lat_df.to_csv('lat_indextrue_df.csv', index=True)

In [45]:
lng_df.to_csv('lng_indextrue_df.csv', index=True)

In [19]:
search = zp.SearchEngine()
x = len(lat)
zlist ={}
for i in range(0,x):
    ilat = lat[i]
    ilng = lng[i]
    zippy = search.by_coordinates(ilat, ilng, radius=10, returns=1)
    zcode = zippy[0].zipcode
    zlist[i] = zcode

In [21]:
len(zlist)

411507

In [26]:
check = pd.DataFrame(zlist , index=[0])

,0,1,2,3,4,5,6,7,8,9,...,411497,411498,411499,411500,411501,411502,411503,411504,411505,411506
0,11218,10098,11238,11385,11215,10454,11222,10020,10003,11222,...,11223,10013,11237,10013,10024,10010,10069,10035,10010,11237


In [28]:
zip_df = check.T

In [30]:
zip_df = zip_df.rename(columns={0:'zipcode'})

In [46]:
zip_df.to_csv('abnb_zip_indextrue.csv',index=True)

In [ ]:
df1['geom'] = df1['latitude'] + ',' + df1['longitude']

In [37]:
df1 = pd.merge(df1, zip_df, left_index=True, right_index=True)

In [38]:
df1 = df1.drop(columns = 'zipcode_x')

In [39]:
df1=df1.rename(columns = {'zipcode_y':'zipcode'})

In [50]:
df1.to_csv('abnb_clean.gz',index=True)

In [47]:
tester_df = pd.read_csv('abnb_zip_indextrue.csv', low_memory=False)


In [48]:
tester_df

,Unnamed: 0,zipcode
0,0,11218
1,1,10098
2,2,11238
3,3,11385
4,4,11215
...,...,...
411502,411502,10010
411503,411503,10069
411504,411504,10035
411505,411505,10010
